In [12]:
from collections import Counter

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd
from urllib.parse import urlparse
import re
import requests
from google.colab import userdata

from bs4 import BeautifulSoup, SoupStrainer
from urllib.request import urlopen

In [13]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [14]:
# Mine the Web for Content

url = 'https://data.usajobs.gov/api/search'
headers = {
    'Host': 'data.usajobs.gov',
    'User-Agent': 'sgus@uw.edu',
    'Authorization-Key': userdata.get('usajobs')
}
params = {
    'Keyword' : 'Business Intelligence',
    'LocationName' : '',
    'ResultsPerPage' : '50'
}

In [15]:
response = requests.get(url, headers=headers, params=params)

if response.status_code == requests.codes.ok:
    json_response = response.json()
    # Use the JSON response as needed
else:
    response.raise_for_status()  # Raise an exception for unsuccessful responses

print('Count of results:',json_response['SearchResult']['SearchResultCountAll'])

Count of results: 57


In [16]:
print('Example record:\n')
json_response['SearchResult']['SearchResultItems'][0]

Example record:



{'MatchedObjectId': '830050000',
 'MatchedObjectDescriptor': {'PositionID': 'MCAD256418072160',
  'PositionTitle': 'Supervisory Procurement Analyst',
  'PositionURI': 'https://www.usajobs.gov:443/GetJob/ViewDetails/830050000',
  'ApplyURI': ['https://www.usajobs.gov:443/GetJob/ViewDetails/830050000'],
  'PositionLocationDisplay': 'Fort Cavazos, Texas',
  'PositionLocation': [{'LocationName': 'Fort Cavazos, Texas',
    'CountryCode': 'United States',
    'CountrySubDivisionCode': 'Texas',
    'CityName': 'Fort Cavazos, Texas',
    'Longitude': -97.7092,
    'Latitude': 31.2014}],
  'OrganizationName': 'U.S. Army Contracting Command',
  'DepartmentName': 'Department of the Army',
  'SubAgency': '418th Contracting Support Brigade (CSB) - Fort Cavazos',
  'JobCategory': [{'Name': 'Contracting', 'Code': '1102'}],
  'JobGrade': [{'Code': 'GS'}],
  'PositionSchedule': [{'Name': '', 'Code': '1'}],
  'PositionOfferingType': [{'Name': '', 'Code': '15317'}],
  'QualificationSummary': "Who May App

In [17]:
# Use beautfulsoup to get and parse webpage
url = "https://www.geeksforgeeks.org/7-skills-needed-to-become-a-business-intelligence-analyst/"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

page = dict()
page['title']=soup.title.text
page

{'title': '7 Skills Needed to Become a Business Intelligence Analyst - GeeksforGeeks'}

In [18]:
# Look at HTML structure to get articles - specific to some HTML content structure, not all
articles = soup.find_all("article")
sections = []
for i,a in enumerate(articles):
  divTag = a.find_all("div")
  for tag in divTag:
    for element in tag.find_all("p"):
      sections.append(element.text)
page['sections']=sections

print(page['title'])
print(' '.join(page['sections']))

sentences = sent_tokenize(' '.join(page['sections']))

7 Skills Needed to Become a Business Intelligence Analyst - GeeksforGeeks
What is the need of the hour for businesses these days? You guessed it right, it’s business intelligence! BI is one of the fundamental components for businesses if they want to remain profitable and increase their brand awareness well beyond their competitors. After all, business is essentially a competition where business intelligence can provide an edge to companies. Keeping this in mind, jobs like Business Intelligence Analyst are soaring in popularity! That’s where you come in! You can take advantage of this rising popularity of Business Intelligence to bag a BI Analyst job that will teach you a lot and take your career to new heights.  But what skills are needed to become a Business Intelligence Analyst?   Well, you need to have a diverse range of business, statistical, and analytical skills to become a Business Intelligence Analyst. You should be well versed in technical aspects such as data analytics and v

In [19]:
# Find links for web mining to follow that are not on the origin site

origin_host = urlparse(url)
print(origin_host.hostname,'-->')
for link in BeautifulSoup(html, 'html.parser', parse_only=SoupStrainer('a')):
    if link.has_attr('href'):
        parsed = urlparse(link['href'])
        if parsed.hostname != origin_host.hostname:
          print('...',link['href'])

www.geeksforgeeks.org -->
... #main
... https://practice.geeksforgeeks.org/company-tags/?ref=outind
... https://practice.geeksforgeeks.org/explore?page=1&sortBy=submissions
... https://geeksforgeeksapp.page.link/?link=https://www.geeksforgeeks.org/7-skills-needed-to-become-a-business-intelligence-analyst/?type%3Darticle%26id%3D521233&apn=free.programming.programming&isi=1641848816&ibi=org.geeksforgeeks.GeeksforGeeksDev&efr=1
... https://news.google.com/publications/CAAqBwgKMLTrzwsw44bnAw?hl=en-IN&gl=IN&ceid=IN%3Aen
... https://news.google.com/publications/CAAqBwgKMLTrzwsw44bnAw?hl=en-IN&gl=IN&ceid=IN%3Aen
... https://www.facebook.com/geeksforgeeks.org/
... https://www.instagram.com/geeks_for_geeks/
... https://in.linkedin.com/company/geeksforgeeks
... https://twitter.com/geeksforgeeks
... https://www.youtube.com/geeksforgeeksvideos
... https://geeksforgeeksapp.page.link/gfg-app
... https://geeksforgeeksapp.page.link/gfg-app
... #
... https://write.geeksforgeeks.org/posts-new?cid=e8fc46

In [20]:
# Mini project - Book store personalized recommendations from reading lists

url = 'https://lithub.com/here-are-haruki-murakamis-five-favorite-books/'

In [21]:
# Use beautfulsoup to get and parse webpage
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

page = dict()
page['title']=soup.title.text
page
content = dict()


In [34]:
text = soup.get_text()
print(text)







Here are Haruki Murakami’s five favorite books. ‹ Literary  Hub


































































































































Craft and Criticism
Fiction and Poetry
News and Culture
Lit Hub Radio
Reading Lists
Book Marks
CrimeReads
About
Log In
 




























								Literary Hub
							













Craft and Criticism Literary Criticism
Craft and Advice
In Conversation
On Translation

Fiction and Poetry Short Story
From the Novel
Poem

News and Culture History
Science
Politics
Biography
Memoir
Food
Technology
Bookstores and Libraries
Film and TV
Travel
Music
Art and Photography
The Hub
Style
Design
Sports
Freeman’s
The Virtual Book Channel

Lit Hub Radio The Lit Hub Podcast
The Critic and Her Publics
Awakeners
Fiction/Non/Fiction
I’m a Writer But
Windham-Campbell Prizes Podcast
Write-minded
First Draft: A Dialogue on Writing
Behind the Mic
Lit Century
Tor Presents: Voyage Into Genre
Beyond the Pa

In [57]:
all_paragraphs = soup.find_all('p')
next = 0
for paragraph in all_paragraphs:
  if not next:
    if paragraph.get_text().strip() == '*':
      next=1
  else:
    line = paragraph.get_text().split(',')
    author = line[0].strip()
    title = line[1].strip()
    print(author,' by ',title,sep='')
    next = 0


F. Scott Fitzgerald by The Great Gatsby
Raymond Chandler by The Long Goodbye
Franz Kafka by The Castle
Fyodor Dostoevsky by The Brothers Karamazov
J.D. Salinger by The Catcher in the Rye
In addition by on his website


In [2]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 4.9 MB/s eta 0:00:00


In [6]:
import anthropic
from google.colab import userdata

In [4]:
ANTHROPIC_MODEL = "claude-3-haiku-20240307"

In [7]:
anthropic_client = anthropic.Anthropic(
    api_key=userdata.get('ANTHROPIC_API_KEY')
)

In [9]:
def anthro_llm(message,temperature=0.0,model=ANTHROPIC_MODEL):
  message = anthropic_client.messages.create(
      model=model,
      temperature=temperature,
      max_tokens=1024,
      messages=[
          {"role": "user", "content": message}
      ]
  )
  return message.content[0].text

In [53]:
print(anthro_llm("Please extract the book author and title from this html and return a list of only the author and title, with no other commentary: "+soup.get_text()) )

Here is the list of the author and title extracted from the HTML:

Haruki Murakami
1. The Great Gatsby by F. Scott Fitzgerald
2. The Long Goodbye by Raymond Chandler
3. The Castle by Franz Kafka
4. The Brothers Karamazov by Fyodor Dostoevsky
5. The Catcher in the Rye by J.D. Salinger


In [87]:
def check_author(possible):
  response = requests.get('https://openlibrary.org/search.json?author='+possible+'&sort=new')

  if response.status_code == requests.codes.ok:
      json_response = response.json()
  else:
      response.raise_for_status()  # Raise an exception for unsuccessful responses

  for doc in json_response['docs']:
    if doc['author_name'][0].lower().strip() == possible.lower().strip():
      print(doc['author_name'][0].lower())


In [88]:
print(check_author('Haruki%20Murakami'))

None
